<a href="https://colab.research.google.com/github/alexgaaranes/malaia-group-2/blob/main/MALAIA_Liyab_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MALAIA - Group 2

---
##### Predicting Starting Salaries of Filipino Graduates Using Academic Background and Industry Placement: A Machine Learning Approach Based on the Liyab First Pay Survey

<br>

Cleaning data from [**Liyab First Pay Survey dataset**](https://docs.google.com/spreadsheets/d/1gnA91Tjr_3UCNV8x1_LoE0oC56r-pXXRdJcgTfOLlm0/edit?gid=549575995#gid=549575995)

### Data Prep and Initial Exploration

In [18]:
# Mounting Google Drive. If running locally, ensure 'liyab.csv' is in the same directory or provide the correct path.
try:
    from google.colab import drive
    drive.mount('/content/drive')
    # Update this path if your file is located elsewhere in Google Drive
    csv_path = "/content/drive/Shareddrives/MALAIA Group 2/liyab_data/liyab.csv"
except ModuleNotFoundError:
    print("Not running in Colab. Assuming 'liyab.csv' is in the current directory or accessible via a local path.")
    csv_path = "liyab.csv" # Adjust if your local path is different

In [19]:
import pandas as pd
import numpy as np
import re

# Read data
try:
    liyab = pd.read_csv(csv_path)
except FileNotFoundError:
    print(f"Error: The file {csv_path} was not found. Please check the path.")
    liyab = pd.DataFrame()

if not liyab.empty:
  print(f"Successfully loaded data. Shape: {liyab.shape}")

Successfully loaded data. Shape: (2933, 9)


#### 1. Year Started First Job

In [ ]:
print("Initial Data Information:")
if not liyab.empty:
    liyab.info()
    print("
Missing Values per Column:")
    print(liyab.isnull().sum())
    print("
First 5 Rows:")
    print(liyab.head())
else:
    print("DataFrame is empty. Cannot display info.")

### Data Cleaning

#### 1. Year Started First Job

In [ ]:
year_col = 'What year did you start your first job?'
if not liyab.empty and year_col in liyab.columns:
    liyab[year_col] = pd.to_numeric(liyab[year_col], errors='coerce')
    original_rows = len(liyab)
    liyab.dropna(subset=[year_col], inplace=True)
    liyab = liyab[liyab[year_col].between(1987, 2025)]
    liyab[year_col] = liyab[year_col].astype(int)
    print(f"Rows removed due to invalid/outside range year: {original_rows - len(liyab)}")
    print(f"Cleaned value counts for '{year_col}':")
    print(liyab[year_col].value_counts().sort_index())


#### 2. Gender Cleaning

In [24]:
gender_col = 'What is your gender?'
if not liyab.empty and gender_col in liyab.columns:
    liyab['Cleaned Gender'] = liyab[gender_col].astype(str).str.lower().str.strip()
    gender_map = {
        'female': 'Female', 'f': 'Female', 'femaile': 'Female', 'femail': 'Female', 'femali': 'Female',
        'femalen': 'Female', 'femal': 'Female', 'femalr': 'Female', 'femalw': 'Female', 'femaled': 'Female',
        'femae': 'Female', 'feme': 'Female', 'babae': 'Female', 'cisgender female': 'Female', 'cis female': 'Female',
        'women': 'Female', 'woman': 'Female', 'female (cishet)': 'Female', 'biological female': 'Female',
        'heterosexual female': 'Female', 'female (queer)': 'Female', 'cisgender-female': 'Female',
        'female, cisgender': 'Female', 'cis woman/female': 'Female', 'frmale': 'Female', '*sex = female': 'Female',
        'male': 'Male', 'm': 'Male', 'make': 'Male', 'man': 'Male', 'cisgender male': 'Male', 'cis male': 'Male',
        'male cisgender': 'Male', 'heterosexual male': 'Male', 'homosexual man': 'Male', 'males': 'Male',
        'mqle': 'Male', 'norzagaray collegemale': 'Male', 'homosexual male': 'Male',
        'lgbtq': 'LGBTQ+', 'gay': 'LGBTQ+', 'lesbian': 'LGBTQ+', 'queer': 'LGBTQ+', 'bisexual': 'LGBTQ+',
        'bisexual woman': 'LGBTQ+', 'bisexual female': 'LGBTQ+', 'cis-gender, pansexual, masculine': 'LGBTQ+',
        'nonbinary': 'LGBTQ+', 'non-binary': 'LGBTQ+', 'nb': 'LGBTQ+', 'gender fluid': 'LGBTQ+',
        'non-conforming': 'LGBTQ+', 'non-conforming male': 'LGBTQ+', 'non-binary, presenting mainly as male': 'LGBTQ+',
        'homosexual': 'LGBTQ+',
        'prefer not to say': 'Prefer not to say', 'prefer not to mention': 'Prefer not to say',
        'tired potato': 'Other', 'pogi': 'Other'
    }
    liyab['Cleaned Gender'] = liyab['Cleaned Gender'].map(gender_map).fillna(liyab['Cleaned Gender'])
    allowed_genders = ['Female', 'Male', 'LGBTQ+', 'Prefer not to say', 'Other']
    liyab['Cleaned Gender'] = liyab['Cleaned Gender'].apply(lambda x: x if x in allowed_genders else 'Other')
    print("
Cleaned value counts for 'Cleaned Gender':")
    print(liyab['Cleaned Gender'].value_counts())


#### 3. University Cleaning (Refined)

#### 5. Other Date Column Validation (Placeholder)

In [26]:
uni_col = 'What school did you graduate from?'
if not liyab.empty and uni_col in liyab.columns:
    liyab['Cleaned University'] = liyab[uni_col].astype(str).str.lower().str.strip()
    liyab['Cleaned University'] = liyab['Cleaned University'].apply(lambda x: re.sub(r'\s*\([^)]*\)\s*', '', x).strip())
    liyab['Cleaned University'] = liyab['Cleaned University'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    liyab['Cleaned University'] = liyab['Cleaned University'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
    university_map = {
        r'.*university philippines diliman.*': 'University of the Philippines Diliman', r'^upd$': 'University of the Philippines Diliman',
        r'.*university philippines los banos.*': 'University of the Philippines Los Baños', r'^uplb$': 'University of the Philippines Los Baños',
        r'.*university philippines manila.*': 'University of the Philippines Manila', r'^upm$': 'University of the Philippines Manila',
        r'.*university philippines visayas.*': 'University of the Philippines Visayas', r'^upv$': 'University of the Philippines Visayas',
        r'.*university philippines cebu.*': 'University of the Philippines Cebu',
        r'.*university philippines baguio.*': 'University of the Philippines Baguio', r'^upb$': 'University of the Philippines Baguio',
        r'.*university philippines mindanao.*': 'University of the Philippines Mindanao',
        r'.*university philippines.*': 'University of the Philippines (Unspecified Campus)', r'^up$': 'University of the Philippines (Unspecified Campus)',
        r'.*ateneo manila university.*': 'Ateneo de Manila University', r'^admu$': 'Ateneo de Manila University',
        r'.*ateneo davao university.*': 'Ateneo de Davao University', r'^addu$': 'Ateneo de Davao University',
        r'.*ateneo zamboanga university.*': 'Ateneo de Zamboanga University', r'^adzu$': 'Ateneo de Zamboanga University',
        r'.*ateneo naga university.*': 'Ateneo de Naga University', r'^adnu$': 'Ateneo de Naga University',
        r'.*xavier university.*': 'Xavier University - Ateneo de Cagayan', r'.*ateneo cagayan.*': 'Xavier University - Ateneo de Cagayan',
        r'.*ateneo.*': 'Ateneo de Manila University',
        r'.*de la salle university manila.*': 'De La Salle University Manila', r'^dlsu$': 'De La Salle University Manila',
        r'.*de la salle college saint benilde.*': 'De La Salle-College of Saint Benilde', r'^csb$': 'De La Salle-College of Saint Benilde', r'^benilde$': 'De La Salle-College of Saint Benilde',
        r'.*de la salle lipa.*': 'De La Salle Lipa', r'^dlsl$': 'De La Salle Lipa',
        r'.*de la salle university dasmarinas.*': 'De La Salle University Dasmariñas', r'^dlsud$': 'De La Salle University Dasmariñas',
        r'.*de la salle.*': 'De La Salle University Manila',
        r'.*university santo tomas.*': 'University of Santo Tomas', r'^ust$': 'University of Santo Tomas',
        r'.*mapua.*': 'Mapúa University',
        r'.*polytechnic university philippines.*': 'Polytechnic University of the Philippines', r'^pup$': 'Polytechnic University of the Philippines',
        r'.*adamson university.*': 'Adamson University',
        r'.*far eastern university.*': 'Far Eastern University', r'^feu$': 'Far Eastern University',
        r'.*lyceum philippines university.*': 'Lyceum of the Philippines University', r'^lpu$': 'Lyceum of the Philippines University',
        r'.*national university.*': 'National University', r'^nu$': 'National University',
        r'.*pamantasan lungsod maynila.*': 'Pamantasan ng Lungsod ng Maynila', r'^plm$': 'Pamantasan ng Lungsod ng Maynila',
        r'.*san beda.*': 'San Beda University',
        r'.*silliman university.*': 'Silliman University',
        r'.*technological institute philippines.*': 'Technological Institute of the Philippines', r'^tip$': 'Technological Institute of the Philippines',
        r'.*technological university philippines.*': 'Technological University of the Philippines', r'^tup$': 'Technological University of the Philippines',
        r'.*university east.*': 'University of the East', r'^ue$': 'University of the East',
        r'.*university san carlos.*': 'University of San Carlos', r'^usc$': 'University of San Carlos',
        r'.*saint louis university.*': 'Saint Louis University Baguio', r'^slu$': 'Saint Louis University Baguio',
        r'.*cebu institute technology.*': 'Cebu Institute of Technology - University', '^cit.*': 'Cebu Institute of Technology - University',
        r'.*holy angel university.*': 'Holy Angel University', r'^hau$': 'Holy Angel University',
        r'.*university rizal system.*': 'University of Rizal System', r'^urs$': 'University of Rizal System',
        r'\bstill in school\b': 'Still Enrolled', r'\bnot yet a graduate\b': 'Still Enrolled', r'\bnot yet graduated\b': 'Still Enrolled',
        r'\bundergrad\b': 'Still Enrolled / Did Not Graduate',
        r'\bdidnt graduate\b': 'Did Not Graduate', r'\bcollege dropout\b': 'Did Not Graduate',
        r'\bhigh school\b': 'High School Graduate', r'\bhs grad\b': 'High School Graduate',
        r'^na$': 'Not Applicable', r'^n a$': 'Not Applicable', r'nan': 'Not Specified',
        r'.*prefer not to say.*': 'Prefer Not to Say',
        r'\bvocational\b': 'Vocational Graduate',
        r'^\d{4}$': 'Invalid Entry (Year)',
        r'^\s*$': 'Not Specified'
    }
    temp_uni_values = liyab['Cleaned University'].copy()
    for pattern, standard_name in university_map.items():
        mask = temp_uni_values.str.contains(pattern, regex=True, case=False, na=False)
        temp_uni_values[mask] = standard_name
    liyab['Cleaned University'] = temp_uni_values
    known_and_special_categories = set(university_map.values())
    def categorize_remaining(uni_name_str):
        if uni_name_str in known_and_special_categories: return uni_name_str
        if pd.isna(uni_name_str) or uni_name_str.strip() == '': return 'Not Specified'
        keywords = ['university', 'college', 'institute', 'polytechnic', 'academy', 'school']
        if any(keyword in uni_name_str for keyword in keywords) and len(uni_name_str) > 3:
            return 'UNIDENTIFIED_OR_VAGUE_UNIVERSITY'
        return 'UNIDENTIFIED_OR_VAGUE_UNIVERSITY'
    liyab['Cleaned University'] = liyab['Cleaned University'].apply(categorize_remaining)
    liyab['Cleaned University'].fillna('UNIDENTIFIED_OR_VAGUE_UNIVERSITY', inplace=True)
    print(f"Value counts before removing vague entries:
{liyab['Cleaned University'].value_counts()}
")
    rows_before_vague_removal = len(liyab)
    categories_to_remove_strict = ['UNIDENTIFIED_OR_VAGUE_UNIVERSITY', 'Invalid Entry (Year)', 'Not Specified']
    liyab = liyab[~liyab['Cleaned University'].isin(categories_to_remove_strict)]
    print(f"Rows removed due to vague, invalid or unspecified university: {rows_before_vague_removal - len(liyab)}")
    print(f"
Final university value counts:
{liyab['Cleaned University'].value_counts()}
")


#### 4. Industry Cleaning

In [ ]:
!pip install sentence-transformers scikit-learn

#### 5. Job Role Cleaning

In [ ]:
role_col = 'What was your role?'
if not liyab.empty and role_col in liyab.columns:
    liyab['Cleaned Role'] = liyab[role_col].astype(str).str.lower().str.strip()
    role_map = {
        r'.*software engineer.*': 'Software Engineer',
        r'.*developer.*': 'Developer',
        r'.*analyst.*': 'Analyst',
        r'.*consultant.*': 'Consultant',
        r'.*engineer.*': 'Engineer',
        r'.*manager.*': 'Manager',
        r'.*specialist.*': 'Specialist',
        r'.*assistant.*': 'Assistant',
        r'.*associate.*': 'Associate',
        r'.*researcher.*': 'Researcher',
        r'.*teacher.*': 'Teacher',
        r'.*instructor.*': 'Instructor',
        r'.*professor.*': 'Professor',
        r'.*officer.*': 'Officer',
        r'.*designer.*': 'Designer',
        r'.*writer.*': 'Writer',
        r'.*editor.*': 'Editor',
        r'.*planner.*': 'Planner',
        r'.*coordinator.*': 'Coordinator',
        r'.*architect.*': 'Architect',
        r'.*auditor.*': 'Auditor',
        r'.*accountant.*': 'Accountant',
        r'.*supervisor.*': 'Supervisor',
        r'.*lead.*': 'Lead',
        r'.*head.*': 'Head',
        r'.*intern.*': 'Intern',
        r'.*trainee.*': 'Trainee'
    }
    temp_role_values = liyab['Cleaned Role'].copy()
    for pattern, standard_name in role_map.items():
        mask = temp_role_values.str.contains(pattern, regex=True, case=False, na=False)
        temp_role_values[mask] = standard_name
    liyab['Cleaned Role'] = temp_role_values
    print("\nCleaned value counts for 'Cleaned Role':")
    print(liyab['Cleaned Role'].value_counts())


#### 6. Negotiation Cleaning

In [ ]:
negotiate_col = 'Did you negotiate your job offer?'
if not liyab.empty and negotiate_col in liyab.columns:
    liyab['Cleaned Negotiation'] = liyab[negotiate_col].astype(str).str.lower().str.strip()
    negotiate_map = {
        'yes': 'Yes',
        'no': 'No',
        'i tried but they did not budge': 'No'
    }
    liyab['Cleaned Negotiation'] = liyab['Cleaned Negotiation'].map(negotiate_map).fillna('No')
    print("\nCleaned value counts for 'Cleaned Negotiation':")
    print(liyab['Cleaned Negotiation'].value_counts())


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

industry_col = 'In what industry was this job?'
if not liyab.empty and industry_col in liyab.columns:
    def normalize_industry(s):
        if not isinstance(s, str): return ''
        s = s.lower().strip()
        s = re.sub(r'[^a-z0-9\s]', '', s)
        s = re.sub(r'\s+', ' ', s).strip()
        return s
    master_categories = [
        "Accountancy, Banking and Finance", "Business Process Outsourcing (BPO)",
        "Business, Consulting and Management", "Charity and Voluntary Work (NGO)",
        "Creative Arts, Design and Media", "Energy and Utilities",
        "Engineering and Manufacturing", "Environment and Agriculture",
        "Healthcare and Pharmaceuticals", "Hospitality, Events, Leisure, Sport and Tourism",
        "Information Technology (IT)", "Law and Legal Services", "Law Enforcement and Security",
        "Marketing, Advertising and Public Relations (PR)", "Property and Construction",
        "Public Services and Administration (Government)", "Recruitment and Human Resources (HR)",
        "Retail and E-commerce", "Sales", "Education and Training", "Transport and Logistics", "Other"
    ]
    model = SentenceTransformer('all-MiniLM-L6-v2')
    category_embeddings = model.encode(master_categories)
    liyab[industry_col].fillna('', inplace=True)
    unique_normalized_entries = liyab[industry_col].apply(normalize_industry).unique()
    unique_normalized_entries = [e for e in unique_normalized_entries if e]
    normalized_entry_to_category = {}
    if unique_normalized_entries:
        entry_embeddings = model.encode(unique_normalized_entries)
        for i, entry_vector in enumerate(entry_embeddings):
            similarities = cosine_similarity([entry_vector], category_embeddings)[0]
            best_category_idx = np.argmax(similarities)
            normalized_entry_to_category[unique_normalized_entries[i]] = master_categories[best_category_idx]
    manual_overrides = {
        'bpo': "Business Process Outsourcing (BPO)", 'call center': "Business Process Outsourcing (BPO)",
        'kpo': "Business Process Outsourcing (BPO)", 'shared services': "Business Process Outsourcing (BPO)",
        'itbpo': "Information Technology (IT)", 'software engineering': "Information Technology (IT)",
        'fintech': "Accountancy, Banking and Finance", 'ecommerce': "Retail and E-commerce",
        'government': "Public Services and Administration (Government)", 'ngo': "Charity and Voluntary Work (NGO)",
        'real estate': "Property and Construction", 'academe': "Education and Training",
        'education': "Education and Training", 'teaching': "Education and Training",
        'architecture': "Property and Construction", 'construction': "Property and Construction",
        'advertising': "Marketing, Advertising and Public Relations (PR)",
        'media': "Creative Arts, Design and Media", 'telecommunications': "Information Technology (IT)",
        'pharmaceutical': "Healthcare and Pharmaceuticals", 'aviation': "Transport and Logistics"
    }
    for norm_key, override_cat in manual_overrides.items():
        normalized_entry_to_category[norm_key] = override_cat
    def map_to_final_industry_category(original_value):
        normalized_val = normalize_industry(original_value)
        if not normalized_val: return 'Not Specified'
        return normalized_entry_to_category.get(normalized_val, 'Other')
    liyab['Cleaned Industry'] = liyab[industry_col].apply(map_to_final_industry_category)
    print("
Cleaned value counts for 'Cleaned Industry':")
    print(liyab['Cleaned Industry'].value_counts(dropna=False))


#### 5. Final Filtering and Column Selection

In [ ]:
if not liyab.empty:
    salary_column_guess = next((col for col in liyab.columns if 'salary' in col.lower() and 'monthly' in col.lower()), None)
    if salary_column_guess:
        print(f"Identified salary column: {salary_column_guess}")
        if liyab[salary_column_guess].dtype == 'object':
            liyab[salary_column_guess] = liyab[salary_column_guess].astype(str).str.replace(r'[^\d.]', '', regex=True)
        liyab[salary_column_guess] = pd.to_numeric(liyab[salary_column_guess], errors='coerce')
        original_rows = len(liyab)
        liyab.dropna(subset=[salary_column_guess], inplace=True)
        print(f"Rows removed due to missing salary data: {original_rows - len(liyab)}")
    else:
        print("Could not identify a salary column. Final filtering may be incomplete.")
    
    if 'Cleaned University' in liyab.columns:
        original_rows = len(liyab)
        uni_categories_to_drop_final = ['Still Enrolled', 'Still Enrolled / Did Not Graduate', 'Did Not Graduate', 
                                  'High School Graduate', 'Not Applicable', 'Prefer Not to Say', 'Vocational Graduate']
        liyab = liyab[~liyab['Cleaned University'].isin(uni_categories_to_drop_final)]
        print(f"Rows removed due to unsuitable university categories (non-grads, etc.): {original_rows - len(liyab)}")

    print(f"
Final shape of the cleaned DataFrame: {liyab.shape}")

    # Final column selection
    columns_to_keep = [col for col in [salary_column_guess, 'Cleaned University', 'Cleaned Industry', 'Cleaned Gender', 'What year did you start your first job?', 'Cleaned Role', 'Cleaned Negotiation'] if col is not None and col in liyab.columns]
    if columns_to_keep:
        liyab_final = liyab[columns_to_keep].copy()
        print("
Final selected DataFrame for modeling (liyab_final):")
        print(liyab_final.head())
        print(f"Shape of liyab_final: {liyab_final.shape}")
    else:
        print("
No columns selected for the final DataFrame.")
        liyab_final = pd.DataFrame()
